In [1]:
# Import of Important Libraries
import sys, os, time, importlib, glob, pdb
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
import OutflowsCGMInteractionTools.AnalysisTools as l
import OutflowsCGMInteractionTools.HalosData as HD
import OutflowsCGMInteractionTools.Config382To465 as c

In [2]:
# Print the Config File
print("Stampede Simulations")
print("The simulations names in the config file are:",c.ListOfStampedeSimulationsNames)
print("The simulations key names in the config file are:",c.ListOfStampedekeysOfTheSimulationsNames)
print("The snapshot numbers in the config file are:",c.ListOfStampedeSnapshotsNumbers)
print("")

# Creates the List of The Simulationes from the Config File and the Relevant Paths for Them
ListOfStampedeSimulationsNames=c.ListOfStampedeSimulationsNames
ListOfStampedeSimulationsPaths=[]
for i in ListOfStampedeSimulationsNames:
    SnapDirect=c.StampedeSnapDirect
    SnapDirect=SnapDirect+i
    ListOfStampedeSimulationsPaths.append(SnapDirect)
print("The simulations paths for Stampede are: ",ListOfStampedeSimulationsPaths)
print("The snapshots numbers for Stampede are: ",c.ListOfStampedeSnapshotsNumbers)
print("")

# This Block Will Create the Full Paths Directories for the Chosen Simulations and Sanpshot Numbers
ListOfSnapshotsNumbersForStampede=c.ListOfStampedeSnapshotsNumbers
ListOfOutputFolderPathsForStampede=[]
for i in ListOfStampedeSimulationsPaths:
    temp=i+"/output"
    ListOfOutputFolderPathsForStampede.append(temp)
ListOfFullPathsForStampede=[]
for i in ListOfOutputFolderPathsForStampede:
    OutputFiles=os.listdir(i)
    for j in ListOfSnapshotsNumbersForStampede:
        option1="snapshot_"+j+".hdf5"
        option2="snapdir_"+j
        if(option1 in OutputFiles):
            FullPath=i+"/"+option1
        if(option2 in OutputFiles):
            FullPath=i+"/"+option2
        ListOfFullPathsForStampede.append(FullPath) 
print("The full paths for Stampede simulations and snapshots are:")
print(ListOfFullPathsForStampede)

Stampede Simulations
The simulations names in the config file are: ['m12i_res7100']
The simulations key names in the config file are: ['m12i']
The snapshot numbers in the config file are: ['382', '440', '441', '442', '443', '444', '445', '446', '447', '448', '449', '450', '451', '452', '453', '454', '455', '456', '457', '458', '459', '460', '461', '462', '463', '464', '465']

The simulations paths for Stampede are:  ['/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12i_res7100']
The snapshots numbers for Stampede are:  ['382', '440', '441', '442', '443', '444', '445', '446', '447', '448', '449', '450', '451', '452', '453', '454', '455', '456', '457', '458', '459', '460', '461', '462', '463', '464', '465']

The full paths for Stampede simulations and snapshots are:
['/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12i_res7100/output/snapdir_382', '/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12i_res7100/output/snapdir_440', '/scratch/projects/xsede/GalaxiesOnF

In [3]:
# Useful functions for the extracting data: Closest star to the center, calculate ISM radius, and find the index of specific particle I.D
def FindTheClosestStarToTheCenter(ListOfRsInThisLimit):
    IndexOfTheStarThatIsClosestTheCenter=0
    ClosestR=ListOfRsInThisLimit[0]
    for i in range(0,len(ListOfRsInThisLimit)):
        if(ListOfRsInThisLimit[i]<=ClosestR):
            ClosestR=ListOfRsInThisLimit[i]
            IndexOfTheStarThatIsClosestTheCenter=i
    return IndexOfTheStarThatIsClosestTheCenter

def ISM(Rvir,ListOfStarsMasses,ListOfCoordinates):# Calculates the ISM: First taking R_vir and multiply by 0.2, then sum all the masses within 
    # this distance, then multiply by 0.5 (by definition we need the half mass radius), then checks what radius contains this mass, then multiply 
    # this radius by 4 ---> this is the ISM radius
    ListOfCoordinates=np.array(ListOfCoordinates)
    ListOfRs=[]
    for i in range(0,len(ListOfCoordinates)):
        Coordinates=ListOfCoordinates[i]
        R=np.sqrt(Coordinates[0]**2+Coordinates[1]**2+Coordinates[2]**2)
        ListOfRs.append(R)
    RvirLimit=0.2*Rvir
    TotalMassInThisLimit=0
    ListOfStarsInThisLimit=[]
    ListOfRsInThisLimit=[]
    #print("Done1")
    for i in range(0,len(ListOfStarsMasses)):
        DistanceOfTheStar=ListOfRs[i]
        if(DistanceOfTheStar<RvirLimit):
            TotalMassInThisLimit=TotalMassInThisLimit+ListOfStarsMasses[i]
            ListOfStarsInThisLimit.append(ListOfStarsMasses[i])
            ListOfRsInThisLimit.append(ListOfRs[i])
    ListOfStarsInThisLimit=np.array(ListOfStarsInThisLimit)
    ListOfRsInThisLimit=np.array(ListOfRsInThisLimit)
    #print("Done2")
    HalfMassInThisLimit=0.5*TotalMassInThisLimit
    #print("Half mass value= ",HalfMassInThisLimit)
    FillTheHalfMassInThisLimit=0
    MaxRInThisLimit=RvirLimit
    CloseEnough=False
    while(CloseEnough==False):
        MaxRInThisLimit=MaxRInThisLimit/1.1
        FillTheHalfMassInThisLimit=0
        for i in range(0,len(ListOfStarsMasses)):
            DistanceOfTheStar=ListOfRs[i]
            if(DistanceOfTheStar<MaxRInThisLimit):
                FillTheHalfMassInThisLimit=FillTheHalfMassInThisLimit+ListOfStarsMasses[i]
        #print("Fill mass till reach the half mass value= ",FillTheHalfMassInThisLimit)
        #print("The difference is= HalfMassInThisLimit-FillTheHalfMassInThisLimit= ",HalfMassInThisLimit-FillTheHalfMassInThisLimit)
        prop=FillTheHalfMassInThisLimit/HalfMassInThisLimit
        #print("The propotion between them= FillTheHalfMassInThisLimit/HalfMassInThisLimit= ",prop)
        if((np.abs(prop)-1)<0.1):
            CloseEnough=True
    return(4*MaxRInThisLimit)

def FindTheIndexOfParticleFromIDsArray(ParticlesIDsArray,ParticleID):
    index=np.where(ParticlesIDsArray == ParticleID)
    return index

In [4]:
# This block will create an array of all the ISM particle in the first snapshot in the relevant range
AllParticelsIDs=np.array([])
ListOfSnapshotsParts=[]
forloopcount=0
ListOfFullPathsForStampedeWithoutTheFirstOne=ListOfFullPathsForStampede[1:]
for SnapDirect in ListOfFullPathsForStampede:
    ListOfSnapshotsParts=[]
    if(SnapDirect.endswith('.hdf5')):
        ListOfSnapshotsParts.append(SnapDirect)
    else:
        ListOfFiles=os.listdir(SnapDirect)
        for i in ListOfFiles:
            if (i.endswith('.hdf5')):
                ListOfSnapshotsParts.append(SnapDirect+'/'+i)# Append only hdf5 files to the snapshots parts list
    SimulationKey=c.ListOfStampedekeysOfTheSimulationsNames[forloopcount]
    Snap1=l.Snapshot(ListOfSnapshotsParts[0],[0,0,0],[0,0,0])# SnapName, not real center coordinates, not real center velocity
    z=Snap1.Redshift()
    print("The redshift of the given halo is:",z)
    snapnum=HD.Snapnum(SimulationKey,z)
    Mvir=HD.HaloMvir(SimulationKey,z)
    Rvir=HD.HaloRvir(SimulationKey,z)
    CenterCoordinates=HD.HaloCenterCoordinates(SimulationKey,z)
    CenterVelocity=HD.HaloCenterVelocity(SimulationKey,z)
    afactor=HD.HaloaFactor(SimulationKey,z)

    # This part intended to merge all particles data from the different snapshot parts (if there are more than 1 part) 
    StarsMasses=[]
    StarsCoordinates=[]
    TotalNumberOfParticles=0
    Masses=[]
    IDS=[]
    Coordinates=[]
    Temperature=[]
    GasR=[]
    Density=[]
    NeutralH=[]
    GasMasses=[]
    BoxesSizes=[]
    SmoothingLengths=[]
    Velocities=[]
    InternalEnergy=[]
    
    for snapshotname in ListOfSnapshotsParts:
        Snap1=l.Snapshot(snapshotname,CenterCoordinates,CenterVelocity,a=afactor)# SnapName, real center coordinates, real center velocity
        
        NumberOfParticlesOfOneSnapshot=Snap1.number_of_particles()# 0-Gas,1-High Resolution Dark Matter,2-Dummy Particles,3-Dummy Particles,4-Stars,5-Black Holes
        TotalNumberOfParticles=TotalNumberOfParticles+NumberOfParticlesOfOneSnapshot[0]

        MassesOfOneSnapshotStars=Snap1.masses(4)# The masses of the stars
        StarsMasses.extend(MassesOfOneSnapshotStars)
        
        CoordinatesOfOneSnapShotStars=Snap1.coords(4)# The coordinates of the stars
        StarsCoordinates.extend(CoordinatesOfOneSnapShotStars)
        
        MassesOfOneSnapshotGasParticles=Snap1.masses(0)# The masses of the gas particles
        GasMasses.extend(MassesOfOneSnapshotGasParticles)
        
        IDSOfOneSnapshotGasParticles=Snap1.IDS(0)# The I.Ds of the gas particles
        IDS.extend(IDSOfOneSnapshotGasParticles)

        CoordinatesOfOneSnapShot=Snap1.coords(0)# The coordinates of the gas particles
        Coordinates.extend(CoordinatesOfOneSnapShot)

        TemperatureOfOneSnapShot=Snap1.Ts(0)# The temperature of the gas particles
        Temperature.extend(TemperatureOfOneSnapShot)

        GasROfOneSnapShot=Snap1.rs(0)# The r of the gas particles
        GasR.extend(GasROfOneSnapShot)

        DensityOfOneSnapShot=Snap1.rhos(0)# The density of the gas particles
        Density.extend(DensityOfOneSnapShot)

        NeutralHOfOneSnapshot=Snap1.neutralHydrogenAbundance(0)# The neutral hydrogen fraction of the gas particles
        NeutralH.extend(NeutralHOfOneSnapshot)
        
        SmoothingLengthOfOneSnapshot=Snap1.SmoothingLength(0)# The smoothinglength of the gas particles
        SmoothingLengths.extend(SmoothingLengthOfOneSnapshot)
        
        VelocitiesOfOneSnapshot=Snap1.velocities(0)# The velocity [Vx,Vy,Vz] in km/s of the gas particles 
        Velocities.extend(VelocitiesOfOneSnapshot)
        
        InternalEnergyOfOneSnapshot=Snap1.InternalEnergy(0)# The internal energy of the gas particles 
        InternalEnergy.extend(InternalEnergyOfOneSnapshot)
    
    ISM_Radius=ISM(Rvir,StarsMasses,StarsCoordinates)
    Coordinates=np.array(Coordinates)
    Temperature=np.array(Temperature)
    GasR=np.array(GasR)
    Density=np.array(Density)
    NeutralH=np.array(NeutralH)
    GasMasses=np.array(GasMasses)
    IDS=np.array(IDS)
    SmoothingLengths=np.array(SmoothingLengths)
    Velocities=np.array(Velocities)
    InternalEnergy=np.array(InternalEnergy)
    TotalNumberOfGasParticlesInISM=0
    
    # Now we want to select only particles that are in the ISM area and build their properties arays
    ISM_Gas_IDS=[]
    ISM_Gas_Coordinates=[]
    ISM_Gas_R=[]
    ISM_Gas_Masses=[]
    ISM_Gas_Density=[]
    ISM_Gas_Temperature=[]
    ISM_Gas_NeutralH=[]
    ISM_Gas_SmoothingLengths=[] 
    ISM_Gas_Velocities=[]
    ISM_Gas_InternalEnergy=[]
    for i in range(0,len(GasR)):
        if(GasR[i]<=ISM_Radius):
            ISM_Gas_IDS.append(IDS[i])
            ISM_Gas_Coordinates.append(Coordinates[i])
            ISM_Gas_R.append(GasR[i])
            ISM_Gas_Masses.append(GasMasses[i])
            ISM_Gas_Density.append(Density[i])
            ISM_Gas_Temperature.append(Temperature[i])
            ISM_Gas_NeutralH.append(NeutralH[i])
            ISM_Gas_SmoothingLengths.append(SmoothingLengths[i])
            ISM_Gas_Velocities.append(np.sqrt((Velocities[i][0])**2+(Velocities[i][1])**2+(Velocities[i][2])**2)) # |V| in km/s
            ISM_Gas_InternalEnergy.append(InternalEnergy[i])
            TotalNumberOfGasParticlesInISM=TotalNumberOfGasParticlesInISM+1
    
    ISM_Gas_IDS=np.array(ISM_Gas_IDS)
    ISM_Gas_Coordinates=np.array(ISM_Gas_Coordinates)
    ISM_Gas_R=np.array(ISM_Gas_R)
    ISM_Gas_Masses=np.array(ISM_Gas_Masses)
    ISM_Gas_Density=np.array(ISM_Gas_Density)
    ISM_Gas_Temperature=np.array(ISM_Gas_Temperature)
    ISM_Gas_NeutralH=np.array(ISM_Gas_NeutralH)
    ISM_Gas_SmoothingLengths=np.array(ISM_Gas_SmoothingLengths)
    ISM_Gas_Velocities=np.array(ISM_Gas_Velocities)
    ISM_Gas_InternalEnergy=np.array(ISM_Gas_InternalEnergy)
    
    print("ISM radius is: ",ISM_Radius)
    print("The total number of gas particles is:",TotalNumberOfParticles)
    print("The total number of gas particles in the ISM is:",TotalNumberOfGasParticlesInISM)
    tempRedshift=[z,snapnum,IDS,Coordinates,GasR,GasMasses,Density,Temperature,NeutralH,SmoothingLengths,Velocities,InternalEnergy]
    tempRedshiftISM=[z,snapnum,ISM_Gas_IDS,ISM_Gas_Coordinates,ISM_Gas_R,ISM_Gas_Masses,ISM_Gas_Density,ISM_Gas_Temperature,ISM_Gas_NeutralH,ISM_Gas_SmoothingLengths,ISM_Gas_Velocities,ISM_Gas_InternalEnergy]
    tempRedshift=np.array(tempRedshift)
    tempRedshiftISM=np.array(tempRedshiftISM)# This is the array of all the ISM particles in the first snapshot (in the relevant range)
    print("#############################################################################################################")
    break

The redshift of the given halo is: 0.49880657709260845
M_vir of the given halo is: 6.10e+11
R_vir of the given halo is: 131.27110796488265
The closet redshift to the given redshift is: 0.49880669
The center coordinates of the given halo is: c= [29573.91853093, 30647.57385591, 32059.65733071]
The velocity of the center of the given halo is: Vc= [-56.08, 78.36, 92.11]
a factor of the given halo is: 0.6671974489251846
loaded PartType4, Masses
loaded PartType4, Coordinates
loaded PartType0, Masses
loaded PartType0, ParticleIDs
loaded PartType0, Coordinates
loaded PartType0, InternalEnergy
loaded PartType0, Density
loaded PartType0, NeutralHydrogenAbundance
loaded PartType0, SmoothingLength
loaded PartType0, Velocities
loaded PartType4, Masses
loaded PartType4, Coordinates
loaded PartType0, Masses
loaded PartType0, ParticleIDs
loaded PartType0, Coordinates
loaded PartType0, InternalEnergy
loaded PartType0, Density
loaded PartType0, NeutralHydrogenAbundance
loaded PartType0, SmoothingLength


/tmp/ipykernel_36364/2935351812.py:139: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tempRedshift=np.array(tempRedshift)
/tmp/ipykernel_36364/2935351812.py:140: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tempRedshiftISM=np.array(tempRedshiftISM)# This is the array of all the ISM particles in the first snapshot (in the relevant range)


In [5]:
# This block purpose is to chose only number of random ISM particles (in order to reduce the calcultions time)
# The format of the array is: [Redshift,Snapnum,IDs,Coordinates,Rs,Masses,Densities,Temeratures,NeutralH,SmoothingLength,Velocities,InternalEnergies]
#                             [   0    ,   1   , 2 ,     3     ,4,   5   ,    6    ,     7     ,    8   ,       9       ,    10    ,       11       ]
ChooseOnly100000FromTempRedshiftISM=[]
ChooseOnly100000FromTempRedshiftISM.append(tempRedshiftISM[0])           #Redshift
ChooseOnly100000FromTempRedshiftISM.append(tempRedshiftISM[1])           #Snapnum
ChooseOnly100000FromTempRedshiftISM.append(tempRedshiftISM[2][0:1000])  #IDs
ChooseOnly100000FromTempRedshiftISM.append(tempRedshiftISM[3][0:1000])  #Coordinates
ChooseOnly100000FromTempRedshiftISM.append(tempRedshiftISM[4][0:1000])  #R
ChooseOnly100000FromTempRedshiftISM.append(tempRedshiftISM[5][0:1000])  #Masses
ChooseOnly100000FromTempRedshiftISM.append(tempRedshiftISM[6][0:1000])  #Density
ChooseOnly100000FromTempRedshiftISM.append(tempRedshiftISM[7][0:1000])  #Tempreture
ChooseOnly100000FromTempRedshiftISM.append(tempRedshiftISM[8][0:1000])  #NeutralH
ChooseOnly100000FromTempRedshiftISM.append(tempRedshiftISM[9][0:1000])  #SmoothingLength
ChooseOnly100000FromTempRedshiftISM.append(tempRedshiftISM[10][0:1000]) #Velocities |V| in km/s
ChooseOnly100000FromTempRedshiftISM.append(tempRedshiftISM[11][0:1000]) #InertialEnergy
ChooseOnly100000FromTempRedshiftISM=np.array(ChooseOnly100000FromTempRedshiftISM)
np.savez_compressed("/home1/08289/tg875885/radial_to_rotating_flows/Aharon/OutflowsCGM/TrackingData/TrackingFrom382To465/"+str(snapnum)+".npz",ChooseOnly100000FromTempRedshiftISM,dtype=float)

/tmp/ipykernel_36364/2757379006.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ChooseOnly100000FromTempRedshiftISM=np.array(ChooseOnly100000FromTempRedshiftISM)


In [6]:
# This Block Will continue from the 2nd snapshot in the relevnat range, for each ISM particle that was chosen it will find it in the next snapshots
# and save the data for each snapshot, the format of the array is:
# [Redshift,Snapnum,IDs,Coordinates,Rs,Masses,Densities,Temeratures,NeutralH,SmoothingLength,Velocities,InternalEnergies]
AllParticelsIDs=np.array([])
ListOfSnapshotsParts=[]
forloopcount=0
for SnapDirect in ListOfFullPathsForStampedeWithoutTheFirstOne:
    ListOfSnapshotsParts=[]
    if(SnapDirect.endswith('.hdf5')):
        ListOfSnapshotsParts.append(SnapDirect)
    else:
        ListOfFiles=os.listdir(SnapDirect)
        for i in ListOfFiles:
            if (i.endswith('.hdf5')):
                ListOfSnapshotsParts.append(SnapDirect+'/'+i)# Append only hdf5 files to the snapshots parts list
    SimulationKey=c.ListOfStampedekeysOfTheSimulationsNames[forloopcount]
    Snap1=l.Snapshot(ListOfSnapshotsParts[0],[0,0,0],[0,0,0])# SnapName, not real center coordinates, not real center velocity
    z=Snap1.Redshift()
    print("The redshift of the given halo is:",z)
    snapnum=HD.Snapnum(SimulationKey,z)
    Mvir=HD.HaloMvir(SimulationKey,z)
    Rvir=HD.HaloRvir(SimulationKey,z)
    CenterCoordinates=HD.HaloCenterCoordinates(SimulationKey,z)
    CenterVelocity=HD.HaloCenterVelocity(SimulationKey,z)
    afactor=HD.HaloaFactor(SimulationKey,z)

    # This part intended to merge all the data from the different snapshot parts (if there are more than 1 part)
    GasIDS=[]
    GasCoordinates=[]
    GasTemperature=[]
    GasR=[]
    GasDensity=[]
    GasNeutralH=[]
    GasMasses=[]
    GasSmoothingLengths=[]
    GasVelocities=[]
    GasInternalEnergy=[]
    StarsIDS=[]
    StarsCoordinates=[]
    #StarsTemperature=[]
    StarsR=[]
    #StarsDensity=[]
    #StarsNeutralH=[]
    StarsMasses=[]
    #StarsSmoothingLengths=[]
    StarsVelocities=[]
    #StarsInternalEnergy=[]
    
    for snapshotname in ListOfSnapshotsParts:
        Snap1=l.Snapshot(snapshotname,CenterCoordinates,CenterVelocity,a=afactor)# SnapName, real center coordinates, real center velocity
        
        GasIDSOfOneSnapshot=Snap1.IDS(0)# The I.Ds of the gas particles
        GasIDS.extend(GasIDSOfOneSnapshot)
        
        GasCoordinatesOfOneSnapShot=Snap1.coords(0)# The coordinates of the gas particles
        GasCoordinates.extend(GasCoordinatesOfOneSnapShot)
        
        GasTemperatureOfOneSnapShot=Snap1.Ts(0)# The temperature of the gas particles
        GasTemperature.extend(GasTemperatureOfOneSnapShot)
        
        GasROfOneSnapShot=Snap1.rs(0)# The r of the gas particles
        GasR.extend(GasROfOneSnapShot)

        GasDensityOfOneSnapShot=Snap1.rhos(0)# The density of the gas particles
        GasDensity.extend(GasDensityOfOneSnapShot)

        GasNeutralHOfOneSnapshot=Snap1.neutralHydrogenAbundance(0)# The neutral hydrogen fraction of the gas particles
        GasNeutralH.extend(GasNeutralHOfOneSnapshot)
        
        GasMassesOfOneSnapshotGasParticles=Snap1.masses(0)# The masses of the gas particles
        GasMasses.extend(GasMassesOfOneSnapshotGasParticles)
        
        GasSmoothingLengthsOfOneSnapshot=Snap1.SmoothingLength(0)# The smoothinglength of the gas particles
        GasSmoothingLengths.extend(GasSmoothingLengthsOfOneSnapshot)

        GasVelocitiesOfOneSnapshot=Snap1.velocities(0)# The velocities [Vx,Vy,Vz] in km/s of the gas particles
        GasVelocities.extend(GasVelocitiesOfOneSnapshot)
        
        GasInternalEnergyOfOneSnapshot=Snap1.InternalEnergy(0)# The internal energy of the gas particles 
        GasInternalEnergy.extend(GasInternalEnergyOfOneSnapshot)
        
        StarsIDSOfOneSnapshot=Snap1.IDS(4)# The I.Ds of the stars particles
        StarsIDS.extend(StarsIDSOfOneSnapshot)
        
        StarsCoordinatesOfOneSnapshot=Snap1.coords(4)# The coordinates of the stars particles
        StarsCoordinates.extend(StarsCoordinatesOfOneSnapshot)
        
        #StarsTemperatureOfOneSnapshot=Snap1.Ts(4)# The temperature of the stars particles
        #StarsTemperature.extend(StarsTemperatureOfOneSnapshot)
        
        StarsROfOneSnapshot=Snap1.rs(4)# The r of the stars particles
        StarsR.extend(StarsROfOneSnapshot)
        
        #StarsDensityOfOneSnapshot=Snap1.rhos(4)# The density of the stars particles
        #StarsDensity.extend(StarsDensityOfOneSnapshot)
        
        #StarsNeutralHOfOneSnapshot=Snap1.neutralHydrogenAbundance(4)# The neutral hydrogen fraction of the stars particles
        #StarsNeutralH.extend(StarsNeutralHOfOneSnapshot)

        StarsMassesOfOneSnapshot=Snap1.masses(4)# The masses of the stars particles
        StarsMasses.extend(StarsMassesOfOneSnapshot)
        
        #StarsSmoothingLengthsOfOneSnapshot=Snap1.SmoothingLength(4)# The smoothinglength of the stars particles
        #StarsSmoothingLengths.extend(StarsSmoothingLengthsOfOneSnapshot)
        
        StarsVelocitiesOfOneSnapshot=Snap1.velocities(4)# The velocities [Vx,Vy,Vz] in km/s of the stars particles
        StarsVelocities.extend(StarsVelocitiesOfOneSnapshot)
        
        #StarsInternalEnergyOfOneSnapshot=Snap1.InternalEnergy(4)# The internal energy of the stars particles
        #StarsInternalEnergy.extend(StarsInternalEnergyOfOneSnapshot)
     
    GasIDS=np.array(GasIDS)
    GasCoordinates=np.array(GasCoordinates)
    GasTemperature=np.array(GasTemperature)
    GasR=np.array(GasR)
    GasDensity=np.array(GasDensity)
    GasNeutralH=np.array(GasNeutralH)
    GasMasses=np.array(GasMasses)
    GasSmoothingLengths=np.array(GasSmoothingLengths)
    GasVelocities=np.array(GasVelocities)
    GasInternalEnergy=np.array(GasInternalEnergy)
    StarsIDS=np.array(StarsIDS)
    StarsCoordinates=np.array(StarsCoordinates)
    #StarsTemperature=np.array(StarsTemperature)
    StarsR=np.array(StarsR)
    #StarsDensity=np.array(StarsDensity)
    #StarsNeutralH=np.array(StarsNeutralH)
    StarsMasses=np.array(StarsMasses)
    #StarsSmoothingLengths=np.array(StarsSmoothingLengths)
    StarsVelocities=np.array(StarsVelocities)
    #StarsInternalEnergy=np.array(StarsInternalEnergy)

    # This part finds all the ISM starting particles from the firstsnapshot in the next snapshot and saves their data
    TempIDs=[]
    TempCoordinates=[]
    TempR=[]
    TempMasses=[]
    TempDensity=[]
    TempTempreture=[]
    TempNeutralH=[]
    TempSmoothingLength=[]
    TempVelocities=[]
    TempInternalEnergy=[]
    for indexofparticleintheoriginalISM in ChooseOnly100000FromTempRedshiftISM[2]:
        GasIndexInTheNextRedShift=FindTheIndexOfParticleFromIDsArray(GasIDS,indexofparticleintheoriginalISM)
        StarsIndexInTheNextRedShift=FindTheIndexOfParticleFromIDsArray(StarsIDS,indexofparticleintheoriginalISM)
        
        if(len(GasIndexInTheNextRedShift[0])>0):
            TempIDs.append(GasIDS[GasIndexInTheNextRedShift[0][0]])
            TempCoordinates.append(GasCoordinates[GasIndexInTheNextRedShift[0][0]])
            TempR.append(GasR[GasIndexInTheNextRedShift[0][0]])
            TempMasses.append(GasMasses[GasIndexInTheNextRedShift[0][0]])
            TempDensity.append(GasDensity[GasIndexInTheNextRedShift[0][0]])
            TempTempreture.append(GasTemperature[GasIndexInTheNextRedShift[0][0]])
            TempNeutralH.append(GasNeutralH[GasIndexInTheNextRedShift[0][0]])
            TempSmoothingLength.append(GasSmoothingLengths[GasIndexInTheNextRedShift[0][0]])
            TempVelocities.append(np.sqrt((GasVelocities[GasIndexInTheNextRedShift[0][0]][0])**2+(GasVelocities[GasIndexInTheNextRedShift][0][1])**2+(GasVelocities[GasIndexInTheNextRedShift[0][0]][2])**2))
            TempInternalEnergy.append(GasInternalEnergy[GasIndexInTheNextRedShift[0][0]])
    
        if(len(StarsIndexInTheNextRedShift[0])>0):
            TempIDs.append(StarsIDS[StarsIndexInTheNextRedShift[0][0]])
            TempCoordinates.append(StarsCoordinates[StarsIndexInTheNextRedShift[0][0]])
            TempR.append(StarsR[StarsIndexInTheNextRedShift[0][0]])
            TempMasses.append(StarsMasses[StarsIndexInTheNextRedShift[0][0]])
            TempDensity.append(None)
            TempTempreture.append(None)
            TempNeutralH.append(None)
            TempSmoothingLength.append(None)
            TempVelocities.append(np.sqrt((StarsVelocities[StarsIndexInTheNextRedShift[0][0]][0])**2+(StarsVelocities[StarsIndexInTheNextRedShift][0][1])**2+(StarsVelocities[StarsIndexInTheNextRedShift[0][0]][2])**2))
            TempInternalEnergy.append(None)
            
    TempIDs=np.array(TempIDs)
    TempCoordinates=np.array(TempCoordinates)
    TempR=np.array(TempR)
    TempMasses=np.array(TempMasses)
    TempDensity=np.array(TempDensity)
    TempTempreture=np.array(TempTempreture)
    TempNeutralH=np.array(TempNeutralH)
    TempSmoothingLength=np.array(TempSmoothingLength)
    TempVelocities=np.array(TempVelocities)
    TempInternalEnergy=np.array(TempInternalEnergy)
    NextSnapshot=[z,snapnum,TempIDs,TempCoordinates,TempR,TempMasses,TempDensity,TempTempreture,TempNeutralH,TempSmoothingLength,TempVelocities,TempInternalEnergy]
    NextSnapshot=np.array(NextSnapshot)
    np.savez_compressed("/home1/08289/tg875885/radial_to_rotating_flows/Aharon/OutflowsCGM/TrackingData/TrackingFrom382To465/"+str(snapnum)+".npz",NextSnapshot,dtype=float)
    print("#############################################################################################################")

The redshift of the given halo is: 0.316561922731615
M_vir of the given halo is: 6.50e+11
R_vir of the given halo is: 150.45247517794056
The closet redshift to the given redshift is: 0.31656179
The center coordinates of the given halo is: c= [29484.53337439, 30755.99241499, 32201.1011555]
The velocity of the center of the given halo is: Vc= [-50.44, 76.19, 96.73]
a factor of the given halo is: 0.7595541717795107
loaded PartType0, ParticleIDs
loaded PartType0, Coordinates
loaded PartType0, InternalEnergy
loaded PartType0, Density
loaded PartType0, NeutralHydrogenAbundance
loaded PartType0, Masses
loaded PartType0, SmoothingLength
loaded PartType0, Velocities
loaded PartType4, ParticleIDs
loaded PartType4, Coordinates
loaded PartType4, Masses
loaded PartType4, Velocities
loaded PartType0, ParticleIDs
loaded PartType0, Coordinates
loaded PartType0, InternalEnergy
loaded PartType0, Density
loaded PartType0, NeutralHydrogenAbundance
loaded PartType0, Masses
loaded PartType0, SmoothingLength

/tmp/ipykernel_36364/3953117420.py:183: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  NextSnapshot=np.array(NextSnapshot)


#############################################################################################################
The redshift of the given halo is: 0.313807534278141
M_vir of the given halo is: 6.50e+11
R_vir of the given halo is: 150.77550922162936
The closet redshift to the given redshift is: 0.31380749
The center coordinates of the given halo is: c= [29483.10257756, 30757.57543599, 32203.39649711]
The velocity of the center of the given halo is: Vc= [-50.34, 76.05, 96.78]
a factor of the given halo is: 0.7611465207889779
loaded PartType0, ParticleIDs
loaded PartType0, Coordinates
loaded PartType0, InternalEnergy
loaded PartType0, Density
loaded PartType0, NeutralHydrogenAbundance
loaded PartType0, Masses
loaded PartType0, SmoothingLength
loaded PartType0, Velocities
loaded PartType4, ParticleIDs
loaded PartType4, Coordinates
loaded PartType4, Masses
loaded PartType4, Velocities
loaded PartType0, ParticleIDs
loaded PartType0, Coordinates
loaded PartType0, InternalEnergy
loaded PartType0

In [7]:
if(len(GasIndexInTheNextRedShift[0])>0):
    TempIDs.append(GasIDS[GasIndexInTheNextRedShift[0][0]])
    TempCoordinates.append(GasCoordinates[GasIndexInTheNextRedShift[0][0]])
    TempR.append(GasR[GasIndexInTheNextRedShift[0][0]])
    TempMasses.append(GasMasses[GasIndexInTheNextRedShift[0][0]])
    TempDensity.append(GasDensity[GasIndexInTheNextRedShift[0][0]])
    TempTempreture.append(GasTemperature[GasIndexInTheNextRedShift[0][0]])
    TempNeutralH.append(GasNeutralH[GasIndexInTheNextRedShift[0][0]])
    TempSmoothingLength.append(GasSmoothingLengths[GasIndexInTheNextRedShift[0][0]])
    TempVelocities.append(np.sqrt((GasVelocities[GasIndexInTheNextRedShift[0][0]][0])**2+(GasVelocities[GasIndexInTheNextRedShift][0][1])**2+(GasVelocities[GasIndexInTheNextRedShift[0][0]][2])**2))
    TempInternalEnergy.append(GasInternalEnergy[GasIndexInTheNextRedShift[0][0]])
    
if(len(StarsIndexInTheNextRedShift[0])>0):
    TempIDs.append(StarsIDS[StarsIndexInTheNextRedShift[0][0]])
    TempCoordinates.append(StarsCoordinates[StarsIndexInTheNextRedShift[0][0]])
    TempR.append(StarsR[StarsIndexInTheNextRedShift[0][0]])
    TempMasses.append(StarsMasses[StarsIndexInTheNextRedShift[0][0]])
    TempDensity.append(None)
    TempTempreture.append(None)
    TempNeutralH.append(None)
    TempSmoothingLength.append(None)
    TempVelocities.append(np.sqrt((StarsVelocities[StarsIndexInTheNextRedShift[0][0]][0])**2+(StarsVelocities[StarsIndexInTheNextRedShift][0][1])**2+(StarsVelocities[StarsIndexInTheNextRedShift[0][0]][2])**2))
    TempInternalEnergy.append(None)

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [ ]:
if(len(IDS[indexinthenextredshift])>0):
            TempIDs.append(IDS[indexinthenextredshift][0])
        else:
            TempIDs.append(None)
        if(len(Coordinates[indexinthenextredshift])>0):
            TempCoordinates.append(Coordinates[indexinthenextredshift][0])
        else:
            TempCoordinates.append(None)
        if(len(GasR[indexinthenextredshift])>0):
            TempR.append(GasR[indexinthenextredshift][0])
        else:
            TempR.append(None)   
        if(len(GasMasses[indexinthenextredshift])>0):
            TempMasses.append(GasMasses[indexinthenextredshift][0])
        else:
            TempMasses.append(None)    
        if(len(Density[indexinthenextredshift])>0):
            TempDensity.append(Density[indexinthenextredshift][0])
        else:
            TempDensity.append(None)  
        if(len(Temperature[indexinthenextredshift])>0):
            TempTempreture.append(Temperature[indexinthenextredshift][0])
        else:
            TempTempreture.append(None)      
        if(len(NeutralH[indexinthenextredshift])>0):
            TempNeutralH.append(NeutralH[indexinthenextredshift][0])
        else:
            TempNeutralH.append(None)          
        if(len(SmoothingLengths[indexinthenextredshift])>0):
            TempSmoothingLength.append(SmoothingLengths[indexinthenextredshift][0])
        else:
            TempSmoothingLength.append(None)       
        if(len(Velocities[indexinthenextredshift])>0):
            TempVelocities.append(np.sqrt((Velocities[indexinthenextredshift][0][0])**2+(Velocities[indexinthenextredshift][0][1])**2+(Velocities[indexinthenextredshift][0][2])**2))
        else:
            TempVelocities.append(None)  
        if(len(InternalEnergy[indexinthenextredshift])>0):
            TempInternalEnergy.append(InternalEnergy[indexinthenextredshift][0])
        else:
            TempInternalEnergy.append(None)      

In [ ]:
lst=[]
for i in range(465,601):
    lst.append(str(i))
print(lst)

In [ ]:
lst=[]
for i in range(382,466):
    lst.append(str(i))
print(lst)

In [ ]:
lst=[]
for i in range(322,382):
    lst.append(str(i))
print(lst)

In [ ]:
lst=[]
for i in range(277,322):
    lst.append(str(i))
print(lst)

In [ ]:
lst=[]
for i in range(242,277):
    lst.append(str(i))
print(lst)

In [ ]:
lst=[]
for i in range(214,242):
    lst.append(str(i))
print(lst)

In [ ]:
x= np.load("/home1/08289/tg875885/radial_to_rotating_flows/Aharon/TrackingFrom214To241/214.0.npz",allow_pickle=True)
print(x['arr_0'][10])